In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
from scipy.stats import zscore
#pd.set_option('display.max_rows', None)
import plotly.graph_objects as go

# 30 genomes community

In [10]:
## opening recovered MAGs
df = pd.read_table('Cami_30g_6s/All_concatenate_together_fixed.tsv')
df.fillna({'Completeness':50, 'Contamination':15,'n_contigs':200,'Longest_contig':100000,'Strain_heterogeneity':100,'pilon_errors':1000}, inplace=True) #'Completeness','Contamination','n_contigs','Longest_contig','pilon_errors']].fillna(50,15,200,100000,1000)
df

,Bin_Id,Reference,Completeness,Contamination,Strain_heterogeneity,n_contigs,Longest_contig,pilon_errors,Category,Assembly
0,MAG_1_S_care_metabat.001,GCF_900198195.1_Tjejuense_V1_genomic,99.66,0.45,0.00,98.0,546168.0,263.0,careful,Spades_careful
1,MAG_2_S_care_concoct.001,GCF_900198195.1_Tjejuense_V1_genomic,99.66,0.45,0.00,34.0,546168.0,42.0,careful,Spades_meta
2,MAG_2_S_meta_concoct.018,GCF_900198195.1_Tjejuense_V1_genomic,99.66,0.46,25.00,70.0,323626.0,366.0,meta,Spades_meta
3,S_meta_all_bin.14,GCF_900198195.1_Tjejuense_V1_genomic,99.66,0.78,40.00,37.0,401391.0,149.0,all,Spades_meta
4,MAG_0_MH_meta_maxbin.001,GCF_900198195.1_Tjejuense_V1_genomic,99.66,1.30,11.11,154.0,217371.0,759.0,meta,Megahit
...,...,...,...,...,...,...,...,...,...,...
187,S_meta_all_bin.2,GCF_000017265.1_ASM1726v1_genomic,100.00,0.16,100.00,26.0,788084.0,177.0,all,Spades_meta
188,MAG_14_S_meta_maxbin.001,GCF_000017265.1_ASM1726v1_genomic,100.00,0.79,0.00,33.0,788084.0,148.0,meta,Spades_meta
189,MAG_12_MH_meta_maxbin.001,GCF_000017265.1_ASM1726v1_genomic,100.00,1.02,0.00,51.0,788084.0,192.0,meta,Megahit
190,MAG_12_MH_care_maxbin.001,GCF_000017265.1_ASM1726v1_genomic,100.00,1.05,0.00,53.0,788201.0,150.0,careful,Megahit


In [11]:
la = df.sort_values(['Reference','pilon_errors'])

In [12]:
## Selecting only which genome were found and the method
df_reference = df[['Reference','Category','Assembly']].copy()
df_reference['Reference'] = df_reference.Reference.str.rsplit()
df_reference = df_reference.explode('Reference')
df_reference['Reference'] = df_reference['Reference'].str.extract('(G.*_[0-9]+\.[0-9]+)')
df_reference=df_reference.rename(columns = {'Reference':'assembly_accession'})
df_reference

,assembly_accession,Category,Assembly
0,GCF_900198195.1,careful,Spades_careful
1,GCF_900198195.1,careful,Spades_meta
2,GCF_900198195.1,meta,Spades_meta
3,GCF_900198195.1,all,Spades_meta
4,GCF_900198195.1,meta,Megahit
...,...,...,...
187,GCF_000017265.1,all,Spades_meta
188,GCF_000017265.1,meta,Spades_meta
189,GCF_000017265.1,meta,Megahit
190,GCF_000017265.1,careful,Megahit


In [13]:
df_category = df.groupby(['Assembly','Category']).median()
df_category


Completeness  Contamination  Strain_heterogeneity  \
Assembly       Category                                                      
Megahit        all             98.280          0.220                   0.0   
               careful         99.340          0.510                   0.0   
               meta            99.430          0.580                   0.0   
Spades_careful all             99.345          0.430                   0.0   
               careful         99.545          0.385                   0.0   
               meta            99.465          0.575                   0.0   
Spades_meta    all             99.660          0.430                   0.0   
               careful         99.660          0.450                   0.0   
               meta            99.660          0.460                   0.0   

                         n_contigs  Longest_contig  pilon_errors  
Assembly       Category                                           
Megahit        all            40.0        459475.0         164.0  
               careful        40.0        411525.0          94.0  
               meta           62.0        362476.0         271.0  
Spades_careful all            26.0        528421.5          81.0  
               careful        43.5        390622.5         123.0  
               meta           48.5        376355.5         242.0  
Spades_meta    all            37.0        362426.0         263.0  
               careful        37.0        390486.0         101.0  
               meta           45.0        348870.0         266.0

## The ***z-score*** (often called standardization) transforms the info into distribution with a mean of 0 and a typical deviation of 1. Each standardized value is computed by subtracting the mean of the corresponding feature then dividing by the quality deviation.

#### df_z_scaled[column] = (df_z_scaled[column] - df_z_scaled[column].mean()) / df_z_scaled[column].std()

In [14]:
# Z-score
#df_norm_z = (df_category[['Completeness','Contamination','Strain_heterogeneity','n_contigs','Longest_contig','pilon_errors']]-df_category[['Completeness','Contamination','Strain_heterogeneity','n_contigs','Longest_contig','pilon_errors']].mean())/df_category[['Completeness','Contamination','Strain_heterogeneity','n_contigs','Longest_contig','pilon_errors']].std()

#sns.heatmap(df_norm, cmap='viridis',linewidths=.5, vmin = -1, vmax = 1)

In [15]:
# Outra fdorma Z-score

df_norm_z = df_category.copy()
df_norm_z[['Completeness','Contamination','Strain_heterogeneity','n_contigs','Longest_contig','pilon_errors']] = zscore(df_category[['Completeness','Contamination','Strain_heterogeneity','n_contigs','Longest_contig','pilon_errors']])

df_norm_z

T:\Programs\Anaconda\lib\site-packages\scipy\stats\stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


Completeness  Contamination  Strain_heterogeneity  \
Assembly       Category                                                      
Megahit        all          -2.698585      -2.238007                   NaN   
               careful      -0.088904       0.597526                   NaN   
               meta          0.132672       1.281965                   NaN   
Spades_careful all          -0.076594      -0.184690                   NaN   
               careful       0.415798      -0.624686                   NaN   
               meta          0.218841       1.233077                   NaN   
Spades_meta    all           0.698924      -0.184690                   NaN   
               careful       0.698924       0.010864                   NaN   
               meta          0.698924       0.108641                   NaN   

                         n_contigs  Longest_contig  pilon_errors  
Assembly       Category                                           
Megahit        all       -0.228625        1.038597     -0.186205  
               careful   -0.228625        0.150386     -1.095577  
               meta       2.153892       -0.758181      1.203836  
Spades_careful all       -1.744773        2.315739     -1.264460  
               careful    0.150411       -0.236805     -0.718837  
               meta       0.691893       -0.501082      0.827096  
Spades_meta    all       -0.553514       -0.759107      1.099907  
               careful   -0.553514       -0.239333     -1.004640  
               meta       0.312856       -1.010214      1.138880

In [16]:
df_norm_z[['Contamination','n_contigs','pilon_errors']] = -1*df_norm_z[['Contamination','n_contigs','pilon_errors']]
df_norm_z['Strain_heterogeneity'].fillna(0,inplace=True)
df_norm_z


Completeness  Contamination  Strain_heterogeneity  \
Assembly       Category                                                      
Megahit        all          -2.698585       2.238007                   0.0   
               careful      -0.088904      -0.597526                   0.0   
               meta          0.132672      -1.281965                   0.0   
Spades_careful all          -0.076594       0.184690                   0.0   
               careful       0.415798       0.624686                   0.0   
               meta          0.218841      -1.233077                   0.0   
Spades_meta    all           0.698924       0.184690                   0.0   
               careful       0.698924      -0.010864                   0.0   
               meta          0.698924      -0.108641                   0.0   

                         n_contigs  Longest_contig  pilon_errors  
Assembly       Category                                           
Megahit        all        0.228625        1.038597      0.186205  
               careful    0.228625        0.150386      1.095577  
               meta      -2.153892       -0.758181     -1.203836  
Spades_careful all        1.744773        2.315739      1.264460  
               careful   -0.150411       -0.236805      0.718837  
               meta      -0.691893       -0.501082     -0.827096  
Spades_meta    all        0.553514       -0.759107     -1.099907  
               careful    0.553514       -0.239333      1.004640  
               meta      -0.312856       -1.010214     -1.138880

In [17]:
df_norm_z = df_norm_z.reset_index()
df_norm_z

,Assembly,Category,Completeness,Contamination,Strain_heterogeneity,n_contigs,Longest_contig,pilon_errors
0,Megahit,all,-2.698585,2.238007,0.0,0.228625,1.038597,0.186205
1,Megahit,careful,-0.088904,-0.597526,0.0,0.228625,0.150386,1.095577
2,Megahit,meta,0.132672,-1.281965,0.0,-2.153892,-0.758181,-1.203836
3,Spades_careful,all,-0.076594,0.184690,0.0,1.744773,2.315739,1.264460
4,Spades_careful,careful,0.415798,0.624686,0.0,-0.150411,-0.236805,0.718837
5,Spades_careful,meta,0.218841,-1.233077,0.0,-0.691893,-0.501082,-0.827096
6,Spades_meta,all,0.698924,0.184690,0.0,0.553514,-0.759107,-1.099907
7,Spades_meta,careful,0.698924,-0.010864,0.0,0.553514,-0.239333,1.004640
8,Spades_meta,meta,0.698924,-0.108641,0.0,-0.312856,-1.010214,-1.138880


In [18]:
df_norm_z_megahit = df_norm_z[df_norm_z['Assembly'] == "Megahit"]
df_norm_z_megahit

,Assembly,Category,Completeness,Contamination,Strain_heterogeneity,n_contigs,Longest_contig,pilon_errors
0,Megahit,all,-2.698585,2.238007,0.0,0.228625,1.038597,0.186205
1,Megahit,careful,-0.088904,-0.597526,0.0,0.228625,0.150386,1.095577
2,Megahit,meta,0.132672,-1.281965,0.0,-2.153892,-0.758181,-1.203836


In [19]:
## Calculating polygon area


area_list = list()
for index in df_norm_z_megahit.index.to_list():
       r = df_norm_z_megahit[['Completeness', 'Contamination',
              'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values
       area=0
       theta = 360/len(r)
       for i in range(0,len(r)-1,1):
              a = (r[i])+3
              b = (r[i+1])+3
              area+= (a*b*np.sin(np.deg2rad(theta))/2)
       area_list.append(area)

area_list

[22.9001452582683, 20.334795099378685, 8.226359253624]

In [20]:
fig = go.Figure()

for n,index in enumerate(df_norm_z_megahit.index.to_list()):
    fig.add_trace(
                go.Scatterpolar(
                                r=df_norm_z_megahit[['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values,
                                theta=['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors'],
                                fill='toself',
                                name="Megahit {} - area {}".format(df_norm_z_megahit['Category'][index], round(area_list[n],2)),
                                showlegend=True,
                                )
                )

fig.update_layout(
    polar=dict(
        radialaxis=dict(
                        visible=True,
                        range=[-3, 3]
                    )
            ),
    title="Megahit Z-score comparison"
)

fig.show()

In [21]:
df_norm_z_spadesc = df_norm_z[df_norm_z['Assembly'] == "Spades_careful"]
df_norm_z_spadesc

,Assembly,Category,Completeness,Contamination,Strain_heterogeneity,n_contigs,Longest_contig,pilon_errors
3,Spades_careful,all,-0.076594,0.184690,0.0,1.744773,2.315739,1.264460
4,Spades_careful,careful,0.415798,0.624686,0.0,-0.150411,-0.236805,0.718837
5,Spades_careful,meta,0.218841,-1.233077,0.0,-0.691893,-0.501082,-0.827096


In [22]:
## Calculating polygon area


area_list = list()
for index in df_norm_z_spadesc.index.to_list():
       r = df_norm_z_spadesc[['Completeness', 'Contamination',
              'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values
       area=0
       theta = 360/len(r)
       for i in range(0,len(r)-1,1):
              a = (r[i])+3
              b = (r[i+1])+3
              area+= (a*b*np.sin(np.deg2rad(theta))/2)
       area_list.append(area)

area_list

[35.0693794151312, 21.630658394617438, 12.605094404113995]

In [23]:
fig2 = go.Figure()

for n,index in enumerate(df_norm_z_spadesc.index.to_list()):
    fig2.add_trace(
                go.Scatterpolar(
                                r=df_norm_z_spadesc[['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values,
                                theta=['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors'],
                                fill='toself',
                                name="Spades_careful {} - area {}".format(df_norm_z_spadesc['Category'][index], round(area_list[n],2)),
                                showlegend=True,
                                )
                )

fig2.update_layout(
    polar=dict(
        radialaxis=dict(
                        visible=True,
                        range=[-3, 3]
                    )
            ),
    title="Spades_careful Z-score comparison"
)

fig2.show()

In [24]:
df_norm_z_meta = df_norm_z[df_norm_z['Assembly'] == "Spades_meta"]
df_norm_z_meta

,Assembly,Category,Completeness,Contamination,Strain_heterogeneity,n_contigs,Longest_contig,pilon_errors
6,Spades_meta,all,0.698924,0.184690,0.0,0.553514,-0.759107,-1.099907
7,Spades_meta,careful,0.698924,-0.010864,0.0,0.553514,-0.239333,1.004640
8,Spades_meta,meta,0.698924,-0.108641,0.0,-0.312856,-1.010214,-1.138880


In [25]:
## Calculating polygon area


area_list = list()
for index in df_norm_z_meta.index.to_list():
       r = df_norm_z_meta[['Completeness', 'Contamination',
              'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values
       area=0
       theta = 360/len(r)
       for i in range(0,len(r)-1,1):
              a = (r[i])+3
              b = (r[i+1])+3
              area+= (a*b*np.sin(np.deg2rad(theta))/2)
       area_list.append(area)

area_list

[19.145867436149302, 22.321840746214562, 15.796518454198734]

In [26]:
fig2 = go.Figure()

for n,index in enumerate(df_norm_z_meta.index.to_list()):
    fig2.add_trace(
                go.Scatterpolar(
                                r=df_norm_z_meta[['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values,
                                theta=['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors'],
                                fill='toself',
                                name="Spades_meta {} - area {}".format(df_norm_z_meta['Category'][index], round(area_list[n],2)),
                                showlegend=True,
                                )
                )

fig2.update_layout(
    polar=dict(
        radialaxis=dict(
                        visible=True,
                        range=[-3, 3]
                    )
            ),
    title="Spades_meta Z-score comparison"
)

fig2.show()

# 100 genomes community 

In [27]:
## Openning CAMISIN Simulated genomes
df_cam = pd.read_table('Cami_100g_6s/selected_genomes_complete_100.tsv', usecols=['assembly_accession','tax_id','organism_name'])
df_cam

,assembly_accession,tax_id,organism_name
0,GCF_000283655.1,862719,Azospirillum lipoferum 4B
1,GCF_000817975.1,1328314,Azotobacter chroococcum NCIMB 8003
2,GCF_000021345.1,395492,Rhizobium leguminosarum bv. trifolii WSM2304
3,GCF_002752655.1,595536,Methylosinus trichosporium OB3b
4,GCF_000010825.1,634452,Acetobacter pasteurianus IFO 3283-01
...,...,...,...
95,GCF_002022225.1,1909732,Tessaracoccus sp. T2.5-30
96,GCF_002022505.1,1933904,Bartonella sp. WD16.2
97,GCF_002105555.1,1945662,Paracoccus contaminans
98,GCF_002240455.1,2003551,Cellulomonas sp. PSBB021


In [28]:
df = pd.read_table('Cami_100g_6s/All_concatenate_together_fixed.tsv')
df.fillna({'Completeness':50, 'Contamination':15,'n_contigs':200,'Longest_contig':100000,'Strain_heterogeneity':100,'pilon_errors':1000}, inplace=True) #'Completeness','Contamination','n_contigs','Longest_contig','pilon_errors']].fillna(50,15,200,100000,1000)
df

,Bin_Id,Reference,Completeness,Contamination,Strain_heterogeneity,n_contigs,Longest_contig,pilon_errors,Category,Assembly
0,S_care_all_bin.19,GCF_900198195.1_Tjejuense_V1_genomic,99.66,0.45,0.00,25.0,917890.0,189.0,all,Spades_careful
1,S_meta_all_bin.2,GCF_900198195.1_Tjejuense_V1_genomic,99.66,0.45,0.00,42.0,473534.0,338.0,all,Spades_meta
2,MAG_11_care_concoct.022,GCF_900198195.1_Tjejuense_V1_genomic,99.66,0.45,0.00,25.0,610353.0,154.0,careful,Spades_meta
3,MAG_16_care_maxbin.001,GCF_900198195.1_Tjejuense_V1_genomic,99.66,0.78,0.00,39.0,915030.0,209.0,careful,Spades_careful
4,MAG_16_meta_maxbin.001,GCF_900198195.1_Tjejuense_V1_genomic,99.66,0.81,28.57,40.0,586658.0,330.0,meta,Spades_careful
...,...,...,...,...,...,...,...,...,...,...
148,MAG_14_care_maxbin.001,GCF_000010825.1_ASM1082v1_genomic GCF_00217377...,99.50,2.24,80.00,104.0,381168.0,50.0,careful,Spades_careful
149,MAG_14_care_maxbin.001,GCF_000010825.1_ASM1082v1_genomic GCF_00217377...,99.50,2.24,80.00,108.0,381409.0,57.0,careful,Spades_meta
150,MAG_1_MH_Scare_maxbin.001,GCF_000010825.1_ASM1082v1_genomic GCF_00217377...,99.50,2.24,80.00,113.0,381168.0,52.0,careful,Megahit
151,S_care_all_bin.8,GCF_000010825.1_ASM1082v1_genomic GCF_00217377...,99.00,0.00,0.00,85.0,274843.0,40.0,all,Spades_careful


In [29]:
la = df.sort_values(['Reference','pilon_errors'])

In [30]:
df_reference = df[['Reference']].copy()
df_reference = df_reference.drop_duplicates()

df_reference['Reference'] = df_reference.Reference.str.rsplit()

df_reference = df_reference.explode('Reference')
df_reference['Reference'] = df_reference['Reference'].str.extract('(G.*_[0-9]+\.[0-9]+)')
df_reference.columns = ['assembly_accession']
df_reference.head()

,assembly_accession
0,GCF_900198195.1
8,GCF_900198195.1
9,GCF_002441975.1
17,GCF_002441975.1
18,GCF_001713455.1


In [31]:
df_check = pd.merge(df_cam,df_reference, how='outer', indicator = True)
df_check[df_check['_merge'] == 'left_only']

,assembly_accession,tax_id,organism_name,_merge
0,GCF_000283655.1,862719,Azospirillum lipoferum 4B,left_only
1,GCF_000817975.1,1328314,Azotobacter chroococcum NCIMB 8003,left_only
2,GCF_000021345.1,395492,Rhizobium leguminosarum bv. trifolii WSM2304,left_only
3,GCF_002752655.1,595536,Methylosinus trichosporium OB3b,left_only
9,GCF_001932635.1,180957,Pectobacterium carotovorum subsp. brasiliense,left_only
...,...,...,...,...
107,GCF_002022225.1,1909732,Tessaracoccus sp. T2.5-30,left_only
108,GCF_002022505.1,1933904,Bartonella sp. WD16.2,left_only
109,GCF_002105555.1,1945662,Paracoccus contaminans,left_only
110,GCF_002240455.1,2003551,Cellulomonas sp. PSBB021,left_only


In [32]:
df_category = df.groupby(['Assembly','Category']).median()
df_category

Completeness  Contamination  Strain_heterogeneity  \
Assembly       Category                                                      
Megahit        all             50.000         15.000               100.000   
               careful         99.430          0.720                 8.330   
               meta            99.500          0.450                20.000   
Spades_careful all             99.005          0.885                 5.555   
               careful         99.430          1.140                10.125   
               meta            99.195          0.610                26.785   
Spades_meta    all             99.510          0.240                 0.000   
               careful         99.590          0.765                16.670   
               meta            99.305          0.440                 0.000   

                         n_contigs  Longest_contig  pilon_errors  
Assembly       Category                                           
Megahit        all           200.0        100000.0        1000.0  
               careful        45.0        381168.0          57.0  
               meta           61.0        365159.0         243.0  
Spades_careful all            73.5        327278.0          97.5  
               careful        60.0        442897.0          70.0  
               meta           74.0        422583.0         256.5  
Spades_meta    all            42.5        472203.5         204.5  
               careful        37.5        581084.5          43.0  
               meta           60.5        422903.0         224.5

In [33]:
# Outra fdorma Z-score

df_norm_z = df_category.copy()
df_norm_z[['Completeness','Contamination','Strain_heterogeneity','n_contigs','Longest_contig','pilon_errors']] = zscore(df_category[['Completeness','Contamination','Strain_heterogeneity','n_contigs','Longest_contig','pilon_errors']])

df_norm_z

Completeness  Contamination  Strain_heterogeneity  \
Assembly       Category                                                      
Megahit        all          -2.828257       2.824053              2.707876   
               careful       0.357359      -0.338886             -0.427519   
               meta          0.361870      -0.398690             -0.028370   
Spades_careful all           0.329969      -0.302340             -0.522433   
               careful       0.357359      -0.245859             -0.366125   
               meta          0.342214      -0.363251              0.203698   
Spades_meta    all           0.362514      -0.445204             -0.712431   
               careful       0.367670      -0.328919             -0.142266   
               meta          0.349303      -0.400905             -0.712431   

                         n_contigs  Longest_contig  pilon_errors  
Assembly       Category                                           
Megahit        all        2.730453       -2.355768      2.709343  
               careful   -0.593266       -0.076353     -0.670168  
               meta      -0.250172       -0.206137     -0.003584  
Spades_careful all        0.017869       -0.513237     -0.525025  
               careful   -0.271616        0.424081     -0.623579  
               meta       0.028591        0.259396      0.044797  
Spades_meta    all       -0.646874        0.661667     -0.141560  
               careful   -0.754091        1.544360     -0.720341  
               meta      -0.260894        0.261991     -0.069884

In [34]:
df_norm_z[['Contamination','n_contigs','pilon_errors']] = -1*df_norm_z[['Contamination','n_contigs','pilon_errors']]
df_norm_z['Strain_heterogeneity'].fillna(0,inplace=True)
df_norm_z

Completeness  Contamination  Strain_heterogeneity  \
Assembly       Category                                                      
Megahit        all          -2.828257      -2.824053              2.707876   
               careful       0.357359       0.338886             -0.427519   
               meta          0.361870       0.398690             -0.028370   
Spades_careful all           0.329969       0.302340             -0.522433   
               careful       0.357359       0.245859             -0.366125   
               meta          0.342214       0.363251              0.203698   
Spades_meta    all           0.362514       0.445204             -0.712431   
               careful       0.367670       0.328919             -0.142266   
               meta          0.349303       0.400905             -0.712431   

                         n_contigs  Longest_contig  pilon_errors  
Assembly       Category                                           
Megahit        all       -2.730453       -2.355768     -2.709343  
               careful    0.593266       -0.076353      0.670168  
               meta       0.250172       -0.206137      0.003584  
Spades_careful all       -0.017869       -0.513237      0.525025  
               careful    0.271616        0.424081      0.623579  
               meta      -0.028591        0.259396     -0.044797  
Spades_meta    all        0.646874        0.661667      0.141560  
               careful    0.754091        1.544360      0.720341  
               meta       0.260894        0.261991      0.069884

In [35]:
df_norm_z = df_norm_z.reset_index()
df_norm_z

,Assembly,Category,Completeness,Contamination,Strain_heterogeneity,n_contigs,Longest_contig,pilon_errors
0,Megahit,all,-2.828257,-2.824053,2.707876,-2.730453,-2.355768,-2.709343
1,Megahit,careful,0.357359,0.338886,-0.427519,0.593266,-0.076353,0.670168
2,Megahit,meta,0.361870,0.398690,-0.028370,0.250172,-0.206137,0.003584
3,Spades_careful,all,0.329969,0.302340,-0.522433,-0.017869,-0.513237,0.525025
4,Spades_careful,careful,0.357359,0.245859,-0.366125,0.271616,0.424081,0.623579
5,Spades_careful,meta,0.342214,0.363251,0.203698,-0.028591,0.259396,-0.044797
6,Spades_meta,all,0.362514,0.445204,-0.712431,0.646874,0.661667,0.141560
7,Spades_meta,careful,0.367670,0.328919,-0.142266,0.754091,1.544360,0.720341
8,Spades_meta,meta,0.349303,0.400905,-0.712431,0.260894,0.261991,0.069884


In [36]:
df_norm_z_megahit = df_norm_z[df_norm_z['Assembly'] == "Megahit"]
df_norm_z_megahit

,Assembly,Category,Completeness,Contamination,Strain_heterogeneity,n_contigs,Longest_contig,pilon_errors
0,Megahit,all,-2.828257,-2.824053,2.707876,-2.730453,-2.355768,-2.709343
1,Megahit,careful,0.357359,0.338886,-0.427519,0.593266,-0.076353,0.670168
2,Megahit,meta,0.361870,0.398690,-0.028370,0.250172,-0.206137,0.003584


In [37]:
## Calculating polygon area


area_list = list()
for index in df_norm_z_megahit.index.to_list():
       r = df_norm_z_megahit[['Completeness', 'Contamination',
              'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values
       area=0
       theta = 360/len(r)
       for i in range(0,len(r)-1,1):
              a = (r[i])+3
              b = (r[i+1])+3
              area+= (a*b*np.sin(np.deg2rad(theta))/2)
       area_list.append(area)

area_list

[1.270434839067709, 21.771178847262792, 21.068688312349195]

In [38]:
fig = go.Figure()

for n,index in enumerate(df_norm_z_megahit.index.to_list()):
    fig.add_trace(
                go.Scatterpolar(
                                r=df_norm_z_megahit[['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values,
                                theta=['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors'],
                                fill='toself',
                                name="Megahit {} - area {}".format(df_norm_z_megahit['Category'][index], round(area_list[n],2)),
                                showlegend=True,
                                )
                )

fig.update_layout(
    polar=dict(
        radialaxis=dict(
                        visible=True,
                        range=[-3, 3]
                    )
            ),
    title="Megahit Z-score comparison"
)

fig.show()

In [39]:
df_norm_z_spadesc = df_norm_z[df_norm_z['Assembly'] == "Spades_careful"]
df_norm_z_spadesc

,Assembly,Category,Completeness,Contamination,Strain_heterogeneity,n_contigs,Longest_contig,pilon_errors
3,Spades_careful,all,0.329969,0.302340,-0.522433,-0.017869,-0.513237,0.525025
4,Spades_careful,careful,0.357359,0.245859,-0.366125,0.271616,0.424081,0.623579
5,Spades_careful,meta,0.342214,0.363251,0.203698,-0.028591,0.259396,-0.044797


In [40]:
## Calculating polygon area


area_list = list()
for index in df_norm_z_spadesc.index.to_list():
       r = df_norm_z_spadesc[['Completeness', 'Contamination',
              'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values
       area=0
       theta = 360/len(r)
       for i in range(0,len(r)-1,1):
              a = (r[i])+3
              b = (r[i+1])+3
              area+= (a*b*np.sin(np.deg2rad(theta))/2)
       area_list.append(area)

area_list

[18.510704621879473, 22.375251833056122, 22.019655635920433]

In [41]:
fig2 = go.Figure()

for n,index in enumerate(df_norm_z_spadesc.index.to_list()):
    fig2.add_trace(
                go.Scatterpolar(
                                r=df_norm_z_spadesc[['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values,
                                theta=['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors'],
                                fill='toself',
                                name="Spades_careful {} - area {}".format(df_norm_z_spadesc['Category'][index], round(area_list[n],2)),
                                showlegend=True,
                                )
                )

fig2.update_layout(
    polar=dict(
        radialaxis=dict(
                        visible=True,
                        range=[-3, 3]
                    )
            ),
    title="Spades_careful Z-score comparison"
)

fig2.show()

In [42]:
df_norm_z_meta = df_norm_z[df_norm_z['Assembly'] == "Spades_meta"]
df_norm_z_meta

,Assembly,Category,Completeness,Contamination,Strain_heterogeneity,n_contigs,Longest_contig,pilon_errors
6,Spades_meta,all,0.362514,0.445204,-0.712431,0.646874,0.661667,0.141560
7,Spades_meta,careful,0.367670,0.328919,-0.142266,0.754091,1.544360,0.720341
8,Spades_meta,meta,0.349303,0.400905,-0.712431,0.260894,0.261991,0.069884


In [43]:
## Calculating polygon area


area_list = list()
for index in df_norm_z_meta.index.to_list():
       r = df_norm_z_meta[['Completeness', 'Contamination',
              'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values
       area=0
       theta = 360/len(r)
       for i in range(0,len(r)-1,1):
              a = (r[i])+3
              b = (r[i+1])+3
              area+= (a*b*np.sin(np.deg2rad(theta))/2)
       area_list.append(area)

area_list

[22.80467971276679, 28.327073654700392, 20.47324025265492]

In [44]:
fig2 = go.Figure()

for n,index in enumerate(df_norm_z_meta.index.to_list()):
    fig2.add_trace(
                go.Scatterpolar(
                                r=df_norm_z_meta[['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values,
                                theta=['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors'],
                                fill='toself',
                                name="Spades_meta {} - area {}".format(df_norm_z_meta['Category'][index], round(area_list[n],2)),
                                showlegend=True,
                                )
                )

fig2.update_layout(
    polar=dict(
        radialaxis=dict(
                        visible=True,
                        range=[-3, 3]
                    )
            ),
    title="Spades_meta Z-score comparison"
)

fig2.show()

# 200 genomes

In [45]:
df = pd.read_table('Cami_200g_6s/All_concatenate_together_fixed.tsv')
df.fillna({'Completeness':50, 'Contamination':15,'n_contigs':200,'Longest_contig':100000,'Strain_heterogeneity':100,'pilon_errors':1000}, inplace=True) #'Completeness','Contamination','n_contigs','Longest_contig','pilon_errors']].fillna(50,15,200,100000,1000)
df

,Bin_Id,Reference,Completeness,Contamination,Strain_heterogeneity,n_contigs,Longest_contig,pilon_errors,Category,Assembly
0,Smeta_all_bin.35,GCF_002504165.1_ASM250416v1_genomic,99.09,0.40,0.00,22.0,603512.0,42.0,all,Spades_meta
1,MAG_4_Scareful_concoct.068,GCF_002504165.1_ASM250416v1_genomic,99.09,0.40,0.00,19.0,998781.0,4.0,careful,Spades_meta
2,MAG_15_Smeta_concoct.059,GCF_002504165.1_ASM250416v1_genomic,99.09,0.61,54.55,22.0,603512.0,42.0,meta,Spades_careful
3,MAG_4_Smeta_concoct.057,GCF_002504165.1_ASM250416v1_genomic,99.09,1.22,46.15,21.0,603512.0,42.0,meta,Spades_meta
4,MAG_10_MH_Scare_concoct.016,GCF_002504165.1_ASM250416v1_genomic,99.09,1.36,0.00,25.0,998860.0,7.0,careful,Megahit
...,...,...,...,...,...,...,...,...,...,...
175,S_care_all_bin.32,GCF_000179635.2_ASM17963v2_genomic,99.32,0.06,100.00,124.0,200747.0,56.0,all,Spades_careful
176,MAG_17_Scareful_maxbin.001,GCF_000179635.2_ASM17963v2_genomic,99.32,0.06,100.00,96.0,211304.0,109.0,careful,Spades_careful
177,MAG_10_Scareful_maxbin.001,GCF_000179635.2_ASM17963v2_genomic,99.32,0.06,100.00,84.0,211081.0,41.0,careful,Spades_meta
178,MAG_11_MH_Scare_maxbin.001,GCF_000179635.2_ASM17963v2_genomic,99.32,0.06,100.00,93.0,210846.0,109.0,careful,Megahit


In [46]:
la = df.sort_values(['Reference','pilon_errors'])

In [47]:
df_category = df.groupby(['Assembly','Category']).median()
df_category

Completeness  Contamination  Strain_heterogeneity  \
Assembly       Category                                                      
Megahit        all             98.895          0.640                  0.00   
               careful         99.190          0.515                  0.00   
               meta            98.755          0.805                  0.00   
Spades_careful all             98.550          0.655                 38.54   
               careful         99.070          0.565                  0.00   
               meta            98.990          0.620                  0.00   
Spades_meta    all             99.230          0.395                  0.00   
               careful         99.075          0.530                  0.00   
               meta            99.100          0.590                  0.00   

                         n_contigs  Longest_contig  pilon_errors  
Assembly       Category                                           
Megahit        all            64.0        429659.0         208.0  
               careful        48.0        483814.0          46.5  
               meta           71.5        468169.5         209.0  
Spades_careful all           101.0        265222.0         107.0  
               careful        53.5        448892.0         118.0  
               meta           65.5        351697.5         204.0  
Spades_meta    all            44.5        469412.0         192.0  
               careful        35.5        539253.5          38.5  
               meta           59.0        440075.5         198.5

In [48]:
# Outra fdorma Z-score

df_norm_z = df_category.copy()
df_norm_z[['Completeness','Contamination','Strain_heterogeneity','n_contigs','Longest_contig','pilon_errors']] = zscore(df_category[['Completeness','Contamination','Strain_heterogeneity','n_contigs','Longest_contig','pilon_errors']])

df_norm_z

Completeness  Contamination  Strain_heterogeneity  \
Assembly       Category                                                      
Megahit        all          -0.432114       0.465083             -0.353553   
               careful       1.001964      -0.710689             -0.353553   
               meta         -1.112693       2.017103             -0.353553   
Spades_careful all          -2.109255       0.606176              2.828427   
               careful       0.418610      -0.240380             -0.353553   
               meta          0.029708       0.276960             -0.353553   
Spades_meta    all           1.196415      -1.839431             -0.353553   
               careful       0.442917      -0.569597             -0.353553   
               meta          0.564449      -0.005226             -0.353553   

                         n_contigs  Longest_contig  pilon_errors  
Assembly       Category                                           
Megahit        all        0.207842       -0.042981      0.919750  
               careful   -0.685569        0.672871     -1.508691  
               meta       0.626629        0.466073      0.934787  
Spades_careful all        2.273855       -2.216603     -0.598966  
               careful   -0.378459        0.211252     -0.433561  
               meta       0.291599       -1.073521      0.859603  
Spades_meta    all       -0.881002        0.482497      0.679162  
               careful   -1.383546        1.405702     -1.628986  
               meta      -0.071349        0.094710      0.776901

In [49]:
df_norm_z[['Contamination','n_contigs','pilon_errors']] = -1*df_norm_z[['Contamination','n_contigs','pilon_errors']]
df_norm_z['Strain_heterogeneity'].fillna(0,inplace=True)
df_norm_z

Completeness  Contamination  Strain_heterogeneity  \
Assembly       Category                                                      
Megahit        all          -0.432114      -0.465083             -0.353553   
               careful       1.001964       0.710689             -0.353553   
               meta         -1.112693      -2.017103             -0.353553   
Spades_careful all          -2.109255      -0.606176              2.828427   
               careful       0.418610       0.240380             -0.353553   
               meta          0.029708      -0.276960             -0.353553   
Spades_meta    all           1.196415       1.839431             -0.353553   
               careful       0.442917       0.569597             -0.353553   
               meta          0.564449       0.005226             -0.353553   

                         n_contigs  Longest_contig  pilon_errors  
Assembly       Category                                           
Megahit        all       -0.207842       -0.042981     -0.919750  
               careful    0.685569        0.672871      1.508691  
               meta      -0.626629        0.466073     -0.934787  
Spades_careful all       -2.273855       -2.216603      0.598966  
               careful    0.378459        0.211252      0.433561  
               meta      -0.291599       -1.073521     -0.859603  
Spades_meta    all        0.881002        0.482497     -0.679162  
               careful    1.383546        1.405702      1.628986  
               meta       0.071349        0.094710     -0.776901

In [50]:
df_norm_z = df_norm_z.reset_index()
df_norm_z

,Assembly,Category,Completeness,Contamination,Strain_heterogeneity,n_contigs,Longest_contig,pilon_errors
0,Megahit,all,-0.432114,-0.465083,-0.353553,-0.207842,-0.042981,-0.919750
1,Megahit,careful,1.001964,0.710689,-0.353553,0.685569,0.672871,1.508691
2,Megahit,meta,-1.112693,-2.017103,-0.353553,-0.626629,0.466073,-0.934787
3,Spades_careful,all,-2.109255,-0.606176,2.828427,-2.273855,-2.216603,0.598966
4,Spades_careful,careful,0.418610,0.240380,-0.353553,0.378459,0.211252,0.433561
5,Spades_careful,meta,0.029708,-0.276960,-0.353553,-0.291599,-1.073521,-0.859603
6,Spades_meta,all,1.196415,1.839431,-0.353553,0.881002,0.482497,-0.679162
7,Spades_meta,careful,0.442917,0.569597,-0.353553,1.383546,1.405702,1.628986
8,Spades_meta,meta,0.564449,0.005226,-0.353553,0.071349,0.094710,-0.776901


In [51]:
df_norm_z_megahit = df_norm_z[df_norm_z['Assembly'] == "Megahit"]
df_norm_z_megahit

,Assembly,Category,Completeness,Contamination,Strain_heterogeneity,n_contigs,Longest_contig,pilon_errors
0,Megahit,all,-0.432114,-0.465083,-0.353553,-0.207842,-0.042981,-0.919750
1,Megahit,careful,1.001964,0.710689,-0.353553,0.685569,0.672871,1.508691
2,Megahit,meta,-1.112693,-2.017103,-0.353553,-0.626629,0.466073,-0.934787


In [52]:
## Calculating polygon area


area_list = list()
for index in df_norm_z_megahit.index.to_list():
       r = df_norm_z_megahit[['Completeness', 'Contamination',
              'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values
       area=0
       theta = 360/len(r)
       for i in range(0,len(r)-1,1):
              a = (r[i])+3
              b = (r[i+1])+3
              area+= (a*b*np.sin(np.deg2rad(theta))/2)
       area_list.append(area)

area_list

[15.161938684417064, 27.938111562771265, 11.311014519522374]

In [53]:
fig = go.Figure()

for n,index in enumerate(df_norm_z_megahit.index.to_list()):
    fig.add_trace(
                go.Scatterpolar(
                                r=df_norm_z_megahit[['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values,
                                theta=['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors'],
                                fill='toself',
                                name="Megahit {} - area {}".format(df_norm_z_megahit['Category'][index], round(area_list[n],2)),
                                showlegend=True,
                                )
                )

fig.update_layout(
    polar=dict(
        radialaxis=dict(
                        visible=True,
                        range=[-3, 3]
                    )
            ),
    title="Megahit Z-score comparison"
)

fig.show()

In [54]:
df_norm_z_spadesc = df_norm_z[df_norm_z['Assembly'] == "Spades_careful"]
df_norm_z_spadesc

,Assembly,Category,Completeness,Contamination,Strain_heterogeneity,n_contigs,Longest_contig,pilon_errors
3,Spades_careful,all,-2.109255,-0.606176,2.828427,-2.273855,-2.216603,0.598966
4,Spades_careful,careful,0.418610,0.240380,-0.353553,0.378459,0.211252,0.433561
5,Spades_careful,meta,0.029708,-0.276960,-0.353553,-0.291599,-1.073521,-0.859603


In [55]:
## Calculating polygon area


area_list = list()
for index in df_norm_z_spadesc.index.to_list():
       r = df_norm_z_spadesc[['Completeness', 'Contamination',
              'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values
       area=0
       theta = 360/len(r)
       for i in range(0,len(r)-1,1):
              a = (r[i])+3
              b = (r[i+1])+3
              area+= (a*b*np.sin(np.deg2rad(theta))/2)
       area_list.append(area)

area_list

[10.264598896390076, 21.85376629546781, 13.841312900523135]

In [56]:
fig2 = go.Figure()

for n,index in enumerate(df_norm_z_spadesc.index.to_list()):
    fig2.add_trace(
                go.Scatterpolar(
                                r=df_norm_z_spadesc[['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values,
                                theta=['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors'],
                                fill='toself',
                                name="Spades_careful {} - area {}".format(df_norm_z_spadesc['Category'][index], round(area_list[n],2)),
                                showlegend=True,
                                )
                )

fig2.update_layout(
    polar=dict(
        radialaxis=dict(
                        visible=True,
                        range=[-3, 3]
                    )
            ),
    title="Spades_careful Z-score comparison"
)

fig2.show()

In [57]:
df_norm_z_meta = df_norm_z[df_norm_z['Assembly'] == "Spades_meta"]
df_norm_z_meta

,Assembly,Category,Completeness,Contamination,Strain_heterogeneity,n_contigs,Longest_contig,pilon_errors
6,Spades_meta,all,1.196415,1.839431,-0.353553,0.881002,0.482497,-0.679162
7,Spades_meta,careful,0.442917,0.569597,-0.353553,1.383546,1.405702,1.628986
8,Spades_meta,meta,0.564449,0.005226,-0.353553,0.071349,0.094710,-0.776901


In [58]:
## Calculating polygon area

theta = 360/len(r)
area_list = list()
for index in df_norm_z_meta.index.to_list():
       r = df_norm_z_meta[['Completeness', 'Contamination',
              'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values
       area=0
       for i in range(0,len(r)-1,1):
              a = (r[i])+3
              b = (r[i+1])+3
              area+= (a*b*np.sin(np.deg2rad(theta))/2)
       area_list.append(area)

area_list

[28.139033101218608, 31.62894427150374, 18.696659174220187]

In [59]:
fig2 = go.Figure()

for n,index in enumerate(df_norm_z_meta.index.to_list()):
    fig2.add_trace(
                go.Scatterpolar(
                                r=df_norm_z_meta[['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values,
                                theta=['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors'],
                                fill='toself',
                                name="Spades_meta {} - area {}".format(df_norm_z_meta['Category'][index], round(area_list[n],2)),
                                showlegend=True,
                                )
                )

fig2.update_layout(
    polar=dict(
        radialaxis=dict(
                        visible=True,
                        range=[-3, 3]
                    )
            ),
    title="Spades_meta Z-score comparison"
)

fig2.show()

# Todos juntos

In [60]:
df30 = pd.read_table('Cami_30g_6s/All_concatenate_together_fixed.tsv')
df30.fillna({'Completeness':50, 'Contamination':15,'n_contigs':200,'Longest_contig':100000,'Strain_heterogeneity':100,'pilon_errors':1000}, inplace=True)
df30['Community'] = '30G'
df100 = pd.read_table('Cami_100g_6s/All_concatenate_together_fixed.tsv')
df100.fillna({'Completeness':50, 'Contamination':15,'n_contigs':200,'Longest_contig':100000,'Strain_heterogeneity':100,'pilon_errors':1000}, inplace=True)
df100['Community'] = '100G'
df200 = pd.read_table('Cami_200g_6s/All_concatenate_together_fixed.tsv')
df200.fillna({'Completeness':50, 'Contamination':15,'n_contigs':200,'Longest_contig':100000,'Strain_heterogeneity':100,'pilon_errors':1000}, inplace=True)
df200['Community'] = '200G'


In [61]:
df_all = pd.concat([df30,df100,df200])
all_category = df_all.groupby(['Community','Assembly','Category']).median()
all_category

Completeness  Contamination  \
Community Assembly       Category                                
100G      Megahit        all             50.000         15.000   
                         careful         99.430          0.720   
                         meta            99.500          0.450   
          Spades_careful all             99.005          0.885   
                         careful         99.430          1.140   
                         meta            99.195          0.610   
          Spades_meta    all             99.510          0.240   
                         careful         99.590          0.765   
                         meta            99.305          0.440   
200G      Megahit        all             98.895          0.640   
                         careful         99.190          0.515   
                         meta            98.755          0.805   
          Spades_careful all             98.550          0.655   
                         careful         99.070          0.565   
                         meta            98.990          0.620   
          Spades_meta    all             99.230          0.395   
                         careful         99.075          0.530   
                         meta            99.100          0.590   
30G       Megahit        all             98.280          0.220   
                         careful         99.340          0.510   
                         meta            99.430          0.580   
          Spades_careful all             99.345          0.430   
                         careful         99.545          0.385   
                         meta            99.465          0.575   
          Spades_meta    all             99.660          0.430   
                         careful         99.660          0.450   
                         meta            99.660          0.460   

                                   Strain_heterogeneity  n_contigs  \
Community Assembly       Category                                    
100G      Megahit        all                    100.000      200.0   
                         careful                  8.330       45.0   
                         meta                    20.000       61.0   
          Spades_careful all                      5.555       73.5   
                         careful                 10.125       60.0   
                         meta                    26.785       74.0   
          Spades_meta    all                      0.000       42.5   
                         careful                 16.670       37.5   
                         meta                     0.000       60.5   
200G      Megahit        all                      0.000       64.0   
                         careful                  0.000       48.0   
                         meta                     0.000       71.5   
          Spades_careful all                     38.540      101.0   
                         careful                  0.000       53.5   
                         meta                     0.000       65.5   
          Spades_meta    all                      0.000       44.5   
                         careful                  0.000       35.5   
                         meta                     0.000       59.0   
30G       Megahit        all                      0.000       40.0   
                         careful                  0.000       40.0   
                         meta                     0.000       62.0   
          Spades_careful all                      0.000       26.0   
                         careful                  0.000       43.5   
                         meta                     0.000       48.5   
          Spades_meta    all                      0.000       37.0   
                         careful                  0.000       37.0   
                         meta                     0.000       45.0   

                                   Longest_contig  pilon_errors  
Community Assembly     

In [62]:
all_category_norm_z = all_category.copy()
all_category_norm_z[['Completeness','Contamination','Strain_heterogeneity','n_contigs','Longest_contig','pilon_errors']] = zscore(all_category_norm_z[['Completeness','Contamination','Strain_heterogeneity','n_contigs','Longest_contig','pilon_errors']])

all_category_norm_z

Completeness  Contamination  \
Community Assembly       Category                                
100G      Megahit        all          -5.095725       5.086917   
                         careful       0.216554      -0.137744   
                         meta          0.224077      -0.236530   
          Spades_careful all           0.170878      -0.077375   
                         careful       0.216554       0.015922   
                         meta          0.191298      -0.177990   
          Spades_meta    all           0.225151      -0.313363   
                         careful       0.233749      -0.121280   
                         meta          0.203120      -0.240189   
200G      Megahit        all           0.159057      -0.167014   
                         careful       0.190761      -0.212748   
                         meta          0.144011      -0.106645   
          Spades_careful all           0.121979      -0.161526   
                         careful       0.177864      -0.194455   
                         meta          0.169266      -0.174332   
          Spades_meta    all           0.195059      -0.256653   
                         careful       0.178401      -0.207260   
                         meta          0.181088      -0.185308   
30G       Megahit        all           0.092962      -0.320681   
                         careful       0.206881      -0.214578   
                         meta          0.216554      -0.188966   
          Spades_careful all           0.207419      -0.243847   
                         careful       0.228913      -0.260312   
                         meta          0.220315      -0.190796   
          Spades_meta    all           0.241272      -0.243847   
                         careful       0.241272      -0.236530   
                         meta          0.241272      -0.232871   

                                   Strain_heterogeneity  n_contigs  \
Community Assembly       Category                                    
100G      Megahit        all                   4.500598   4.440143   
                         careful              -0.001992  -0.418531   
                         meta                  0.571208   0.083010   
          Spades_careful all                  -0.138293   0.474838   
                         careful               0.086174   0.051663   
                         meta                  0.904469   0.490511   
          Spades_meta    all                  -0.411140  -0.496897   
                         careful               0.407647  -0.653628   
                         meta                 -0.411140   0.067336   
200G      Megahit        all                  -0.411140   0.177048   
                         careful              -0.411140  -0.324492   
                         meta                 -0.411140   0.412146   
          Spades_careful all                   1.481844   1.336861   
                         careful              -0.411140  -0.152088   
                         meta                 -0.411140   0.224068   
          Spades_meta    all                  -0.411140  -0.434204   
                         careful              -0.411140  -0.716321   
                         meta                 -0.411140   0.020317   
30G       Megahit        all                  -0.411140  -0.575262   
                         careful              -0.411140  -0.575262   
                         meta                 -0.411140   0.114356   
          Spades_careful all                  -0.411140  -1.014110   
                         careful              -0.411140  -0.465550   
                         meta                 -0.411140  -0.308819   
          Spades_meta    all                  -0.411140  -0.669301   
                         careful              -0.411140  -0.669301   
                         meta                 -0.411140  -0.418531   

                                   Longest_contig  pilon_errors  
Community Assembly     

In [63]:
all_category_norm_z[['Contamination','n_contigs','pilon_errors']] = -1*all_category_norm_z[['Contamination','n_contigs','pilon_errors']]
all_category_norm_z['Strain_heterogeneity'].fillna(0,inplace=True)
all_category_norm_z

Completeness  Contamination  \
Community Assembly       Category                                
100G      Megahit        all          -5.095725      -5.086917   
                         careful       0.216554       0.137744   
                         meta          0.224077       0.236530   
          Spades_careful all           0.170878       0.077375   
                         careful       0.216554      -0.015922   
                         meta          0.191298       0.177990   
          Spades_meta    all           0.225151       0.313363   
                         careful       0.233749       0.121280   
                         meta          0.203120       0.240189   
200G      Megahit        all           0.159057       0.167014   
                         careful       0.190761       0.212748   
                         meta          0.144011       0.106645   
          Spades_careful all           0.121979       0.161526   
                         careful       0.177864       0.194455   
                         meta          0.169266       0.174332   
          Spades_meta    all           0.195059       0.256653   
                         careful       0.178401       0.207260   
                         meta          0.181088       0.185308   
30G       Megahit        all           0.092962       0.320681   
                         careful       0.206881       0.214578   
                         meta          0.216554       0.188966   
          Spades_careful all           0.207419       0.243847   
                         careful       0.228913       0.260312   
                         meta          0.220315       0.190796   
          Spades_meta    all           0.241272       0.243847   
                         careful       0.241272       0.236530   
                         meta          0.241272       0.232871   

                                   Strain_heterogeneity  n_contigs  \
Community Assembly       Category                                    
100G      Megahit        all                   4.500598  -4.440143   
                         careful              -0.001992   0.418531   
                         meta                  0.571208  -0.083010   
          Spades_careful all                  -0.138293  -0.474838   
                         careful               0.086174  -0.051663   
                         meta                  0.904469  -0.490511   
          Spades_meta    all                  -0.411140   0.496897   
                         careful               0.407647   0.653628   
                         meta                 -0.411140  -0.067336   
200G      Megahit        all                  -0.411140  -0.177048   
                         careful              -0.411140   0.324492   
                         meta                 -0.411140  -0.412146   
          Spades_careful all                   1.481844  -1.336861   
                         careful              -0.411140   0.152088   
                         meta                 -0.411140  -0.224068   
          Spades_meta    all                  -0.411140   0.434204   
                         careful              -0.411140   0.716321   
                         meta                 -0.411140  -0.020317   
30G       Megahit        all                  -0.411140   0.575262   
                         careful              -0.411140   0.575262   
                         meta                 -0.411140  -0.114356   
          Spades_careful all                  -0.411140   1.014110   
                         careful              -0.411140   0.465550   
                         meta                 -0.411140   0.308819   
          Spades_meta    all                  -0.411140   0.669301   
                         careful              -0.411140   0.669301   
                         meta                 -0.411140   0.418531   

                                   Longest_contig  pilon_errors  
Community Assembly     

In [64]:
all_category_norm_z = all_category_norm_z.reset_index()
all_category_norm_z

,Community,Assembly,Category,Completeness,Contamination,Strain_heterogeneity,n_contigs,Longest_contig,pilon_errors
0,100G,Megahit,all,-5.095725,-5.086917,4.500598,-4.440143,-3.398535,-4.599015
1,100G,Megahit,careful,0.216554,0.137744,-0.001992,0.418531,-0.305787,0.753311
2,100G,Megahit,meta,0.224077,0.236530,0.571208,-0.083010,-0.481880,-0.302397
3,100G,Spades_careful,all,0.170878,0.077375,-0.138293,-0.474838,-0.898558,0.523440
4,100G,Spades_careful,careful,0.216554,-0.015922,0.086174,-0.051663,0.373210,0.679525
5,100G,Spades_careful,meta,0.191298,0.177990,0.904469,-0.490511,0.149763,-0.379021
6,100G,Spades_meta,all,0.225151,0.313363,-0.411140,0.496897,0.695571,-0.083876
7,100G,Spades_meta,careful,0.233749,0.121280,0.407647,0.653628,1.893223,0.832773
8,100G,Spades_meta,meta,0.203120,0.240189,-0.411140,-0.067336,0.153283,-0.197393
9,200G,Megahit,all,0.159057,0.167014,-0.411140,-0.177048,0.227597,-0.103742


In [65]:
## Calculating polygon area

area_list = list()
for index in all_category_norm_z.index.to_list():
       r = all_category_norm_z[['Completeness', 'Contamination',
              'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values
       theta = 360/len(r)
       area=0
       for i in range(0,len(r)-1,1):
              a = (r[i])+3
              b = (r[i+1])+3
              area+= (a*b*np.sin(np.deg2rad(theta))/2)
       area_list.append(area)

print(area_list)

[-9.037081545985146, 21.24834859273057, 20.156108379999814, 16.671677611542677, 21.76498418143713, 21.0046924635312, 22.523883092834044, 30.22972714272474, 19.244943686798745, 19.040115345892104, 23.583347223130957, 19.274455532867798, 16.79053384035785, 21.278178466449337, 16.87098645232132, 22.196233426145277, 26.71568755226586, 19.880338966595023, 22.525849808589843, 21.409298239370894, 17.096559177332363, 26.89650968324899, 20.391103781956556, 18.61156992196636, 19.039635768694414, 20.970432831741583, 18.05582373066063]


In [66]:
fig2 = go.Figure()

for n,index in enumerate(all_category_norm_z.index.to_list()):
    fig2.add_trace(
                go.Scatterpolar(
                                r=all_category_norm_z[['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors']].loc[index].values,
                                theta=['Completeness', 'Contamination',
       'Strain_heterogeneity', 'n_contigs', 'Longest_contig', 'pilon_errors'],
                                fill='toself',
                                name="{}: Spades_careful {} - area {}".format(all_category_norm_z['Community'][index],all_category_norm_z['Category'][index], round(area_list[n],2)),
                                showlegend=True,
                                )
                )

fig2.update_layout(
    polar=dict(
        radialaxis=dict(
                        visible=True,
                        range=[-5, 5]
                    )
            ),
    title="Spades_meta Z-score comparison"
)

fig2.show()